# Cleaning US Census Data

You just got hired as a Data Analyst at the Census Bureau, which collects census data and creates interesting visualizations and insights from it.

The person who had your job before you left you all the data they had for the most recent census. It is in multiple `csv` files. They didn't use pandas, they would just look through these `csv` files manually whenever they wanted to find something. Sometimes they would copy and paste certain numbers into Excel to make charts.

The thought of it makes you shiver. This is not scalable or repeatable.

Your boss wants you to make some scatterplots and histograms by the end of the day. Can you get this data into `pandas` and into reasonable shape so that you can make these histograms?

## Inspect the Data!

1. The first visualization your boss wants you to make is a scatterplot that shows average income in a state vs proportion of women in that state.

   Open some of the census `csv` files that came with the kit you downloaded. How are they named? What kind of information do they hold? Will they help us make this graph?

2. It will be easier to inspect this data once we have it in a DataFrame. You can't even call `.head()` on these `csv`s! How are you supposed to read them?

   Using `glob`, loop through the census files available and load them into DataFrames. Then, concatenate all of those DataFrames together into one DataFrame, called something like `us_census`.

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

files = glob.glob("states*.csv")

df_list = []
for filename in files:
    data = pd.read_csv(filename)
    df_list.append(data)

df = pd.concat(df_list)

print(df.head())

   Unnamed: 0           State  TotalPop Hispanic   White   Black Native  \
0           0    Rhode Island   1053661   13.36%  74.33%   5.68%  0.35%   
1           1  South Carolina   4777576    5.06%  62.89%  28.75%  0.29%   
2           2    South Dakota    843190    3.24%  82.50%   1.42%  9.42%   
3           3       Tennessee   6499615    4.72%  73.49%  18.28%  0.23%   
4           4           Texas  26538614   38.05%  44.69%  11.65%  0.26%   

   Asian Pacific       Income            GenderPop  
0  3.25%   0.04%  $59,125.27       510388M_543273F  
1  1.25%   0.05%  $46,296.81     2322409M_2455167F  
2  1.02%   0.04%  $51,805.41       423477M_419713F  
3  1.41%   0.04%  $47,328.08     3167756M_3331859F  
4  3.67%   0.07%  $55,874.52   13171316M_13367298F  


3. Look at the `.columns` and the `.dtypes` of the `us_census` DataFrame. Are those datatypes going to hinder you as you try to make histograms?

In [34]:
print (df.dtypes)

Unnamed: 0     int64
State         object
TotalPop       int64
Hispanic      object
White         object
Black         object
Native        object
Asian         object
Pacific       object
Income        object
GenderPop     object
dtype: object


4. Look at the `head()` of the DataFrame so that you can understand why some of these `dtypes` are objects instead of integers or floats.

   Start to make a plan for how to convert these columns into the right types for manipulation.

In [52]:
#store the columns with percentage sign in a list
percentage_columns = ['Hispanic' , 'White' , 'Black' , 'Native' , 'Asian' , 'Pacific']
percentage_columns_df = df[percentage_columns]

#iterate through the columns and remove the percebtage sign and change the data type of the column to float
for column_name in percentage_columns_df:
    df[column_name] = df[column_name].replace('[\%,]', '', regex=True)
    df[column_name] = pd.to_numeric(df[column_name])


#the same process for the income column with the dollar sign
df["Income"] = df["Income"].replace('[\$,]', '', regex=True)
df['Income'] = pd.to_numeric(df['Income'])


#the gender pop colum contains both female and male population in on string
#first the string is halved and stored in female and male population columns
name_split = df.GenderPop.str.split("_")
df['male_population'] = name_split.str.get(0)
df['female_population'] = name_split.str.get(1)


#the M and F at the end of the strings are removed and replaced with empty chartacter and columns are 
#converted to numeric type afterwards
df['male_population'] = df['male_population'].replace('[^0-9]', "", regex=True)
df['female_population'] = df['female_population'].replace('[^0-9]', "", regex=True)
df['male_population'] = pd.to_numeric(df['male_population']).astype('Int64')
df['female_population'] = pd.to_numeric(df['female_population']).astype('Int64')


#finally the genderpop colum is dropped from the dataframe
df.drop(columns='GenderPop', inplace=False)

    
print (df.head())
print (df.dtypes)

   Unnamed: 0           State  TotalPop  Hispanic  White  Black  Native  \
0           0    Rhode Island   1053661     13.36  74.33   5.68    0.35   
1           1  South Carolina   4777576      5.06  62.89  28.75    0.29   
2           2    South Dakota    843190      3.24  82.50   1.42    9.42   
3           3       Tennessee   6499615      4.72  73.49  18.28    0.23   
4           4           Texas  26538614     38.05  44.69  11.65    0.26   

   Asian  Pacific    Income            GenderPop  male_population  \
0   3.25     0.04  59125.27      510388M_543273F           510388   
1   1.25     0.05  46296.81    2322409M_2455167F          2322409   
2   1.02     0.04  51805.41      423477M_419713F           423477   
3   1.41     0.04  47328.08    3167756M_3331859F          3167756   
4   3.67     0.07  55874.52  13171316M_13367298F         13171316   

   female_population  
0             543273  
1            2455167  
2             419713  
3            3331859  
4           1336729

## Regex to the Rescue

5. Use regex to turn the `Income` column into a format that is ready for conversion into a numerical type.

6. Look at the `GenderPop` column. We are going to want to separate this into two columns, the `Men` column, and the `Women` column.

   Split the column into those two new columns using `str.split` and separating out those results.

7. Convert both of the columns into numerical datatypes.

   There is still an `M` or an `F` character in each entry! We should remove those before we convert.

8. Now you should have the columns you need to make the graph and make sure your boss does not slam a ruler angrily on your desk because you've wasted your whole day cleaning your data with no results to show!

   Use matplotlib to make a scatterplot!
   
   ```py
   plt.scatter(the_women_column, the_income_column)
   ```
   
   Remember to call `plt.show()` to see the graph!

9. You want to double check your work. You know from experience that these monstrous csv files probably have `nan` values in them! Print out your column with the number of women per state to see.

   We can fill in those `nan`s by using pandas' `.fillna()` function.
   
   You have the `TotalPop` per state, and you have the `Men` per state. As an estimate for the `nan` values in the `Women` column, you could use the `TotalPop` of that state minus the `Men` for that state.
   
   Print out the `Women` column after filling the `nan` values to see if it worked!

10. We forgot to check for duplicates! Use `.duplicated()` on your `census` DataFrame to see if we have duplicate rows in there.

11. Drop those duplicates using the `.drop_duplicates()` function.

12. Make the scatterplot again. Now, it should be perfect! Your job is secure, for now.

## Histogram of Races

13. Now your boss wants you to make a bunch of histograms out of the race data that you have. Look at the `.columns` again to see what the race categories are.

14. Try to make a histogram for each one!

    You will have to get the columns into the numerical format, and those percentage signs will have to go.
    
    Don't forget to fill the `nan` values with something that makes sense! You probably dropped the duplicate rows when making your last graph, but it couldn't hurt to check for duplicates again.

## Get Creative

15. Phew. You've definitely impressed your boss on your first day of work.

    But is there a way you really convey the power of pandas and Python over the drudgery of `csv` and Excel?
    
    Try to make some more interesting graphs to show your boss, and the world! You may need to clean the data even more to do it, or the cleaning you have already done may give you the ease of manipulation you've been searching for.